In [1]:
# Imports
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir
import numpy as np

import pandas as pd
from aequilibrae.project.database_connection import database_connection
from aequilibrae.utils.db_utils import read_and_close

from aequilibrae.transit import Transit
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae.utils.create_example import create_example
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.project import Project


# TODO:
# 1. Figure out the capacity vector and ordering etc.
# 2. Update reference files so tests work
# 3. Build PC, understand how venv's and wsl system works from ground up, ubuntu 22.04, 
# 4. Remove this notebook from git.

# Rewrite TODO's!

# CHANGE DOCUMENTATION - WE HAVE TO ADD THE PRELOADS BEFORE SETTING THE ALGORITHM?

In [2]:
fldr = "/data/aequilibrae/models/coquimbo_updated"
print(fldr)
project = Project.from_path(fldr)

/data/aequilibrae/models/coquimbo_updated


In [3]:
# Build temporary updated coquimbo file

# fldr = "/data/aequilibrae/models/coquimbo_updated"
# project = create_example(fldr, "coquimbo")

# remove(join(fldr, "public_transport.sqlite"))
# dest_path = join(fldr, "gtfs_coquimbo.zip")

# data = Transit(project)

# transit = data.new_gtfs_builder(agency="Lisanco", file_path=dest_path)

# transit.load_date("2016-04-13")

# # Now we execute the map matching to find the real paths.
# # Depending on the GTFS size, this process can be really time-consuming.
# transit.set_allow_map_match(True)
# transit.map_match()

# # Finally, we save our GTFS into our model.
# transit.save_to_disk()

In [4]:
# Attempt to build preload with new pce column

project.network.build_graphs()
project.activate()

g = project.network.graphs["c"]
g.set_skimming(["travel_time"])
g.set_graph("travel_time")
g.set_blocked_centroid_flows(False)
g.graph["capacity"] = 500
g.graph["travel_time"] = g.graph["distance"] / 50

def hr_to_sec(e):
    return int(e * 60 * 60)

start = hr_to_sec(7)
end = hr_to_sec(8)
inclusion_cond = "start"

/home/ubuntu/git/aequilibrae/aequilibrae/project/network/network.py:343: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.read_sql(sql, conn).fillna(value=np.nan)


In [5]:
n_zones = g.centroids.shape[0]
matrix = AequilibraeMatrix()
matrix.create_empty(zones=n_zones, matrix_names=["car"])
matrix.index = g.centroids

matrix.matrices[:, :, 0] = 5  # 5 trips from each OD should cause a little bit of congestion
matrix.computational_view("car")

In [6]:
transit = Transit(project)
preload = transit.build_pt_preload(start, end, inclusion_cond=inclusion_cond)
preload

,link_id,direction,preload
0,33,1,152
1,43,1,152
2,44,1,152
3,48,1,152
4,50,1,152
...,...,...,...
247,34529,-1,168
248,34529,1,152
249,34530,-1,168
250,34530,1,152


In [7]:
assignment = TrafficAssignment()
assignment.set_classes([TrafficClass("car", g, matrix)])

assignment.set_vdf("BPR")
assignment.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
assignment.set_capacity_field("capacity")
assignment.set_time_field("travel_time")
assignment.max_iter = 1  # AON assignment
assignment.add_preload(preload, name="pt")
assignment.set_algorithm("msa")

In [8]:
print(assignment.preloads)
print(assignment.preloads["pt"].sum())

       link_id  direction   pt
0            1         -1  0.0
1            1          1  0.0
2            2          1  0.0
3            3          1  0.0
4           12          1  0.0
...        ...        ...  ...
34541    34940          1  0.0
34542    34941         -1  0.0
34543    34941          1  0.0
34544    34942         -1  0.0
34545    34942          1  0.0

[34546 rows x 3 columns]
39696.0


In [11]:
cols = assignment.preloads.columns.difference(['link_id', 'direction'])
p = assignment.preloads[cols].sum(axis=1).to_numpy()
p.sum()

39696.0

In [10]:
# %%
matrix.close()
project.close()